In [37]:
using JuMP, Gurobi, BenchmarkTools, Base.Test

In [79]:
m = Model()
@variable m x
c = @constraint m x <= 1

x ≤ 1

In [80]:
typeof(c)

JuMP.ConstraintRef{JuMP.Model,JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}}

In [83]:
dump(typeof(c.m.linconstr[c.idx]))

JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}} <: JuMP.AbstractConstraint
  terms::JuMP.GenericAffExpr{Float64,JuMP.Variable}
  lb::Float64
  ub::Float64


In [85]:
cn = c.m.linconstr[c.idx]

x ≤ 1

In [ ]:
cn.

In [93]:
module cj3

using JuMP
import Base: hash, ==, show, &

const Constraint{T} = JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{T, JuMP.Variable}}

struct Conditional
    constraints::Set{Constraint{Float64}}
end

# work-around because JuMP doesn't properly define hash()
function _hash(x::Constraint, h::UInt)
    h = hash(x.lb, h)
    h = hash(x.ub, h)
    h = hash(x.terms.constant, h)
    for v in x.terms.vars
        h = hash(v, h)
    end
    for c in x.terms.coeffs
        h = hash(c, h)
    end
    h
end

function hash(c::Conditional, h::UInt)
    for constraint in c.constraints
        h = _hash(constraint, h)
    end
end

(==)(c1::Conditional, c2::Conditional) = c1.constraints == c2.constraints

# (==)(e1::Constraint, e2::Constraint) = (e1.equality == e2.equality) && (e1.terms == e2.terms)

# Base.convert(::Type{HashableAffExpr}, x::AffExpr) = HashableAffExpr(x)
    
# struct Conditional
#     args::Set{HashableAffExpr}
# end

# const Conditional = Set{Constraint}

# hash(c::Conditional, h::UInt) = hash(c.args, hash(Conditional, h))
# (==)(c1::Conditional, c2::Conditional) = c1.args == c2.args

function simplify(e::JuMP.GenericAffExpr{T, Variable}) where T
    coeffs = Dict{Variable, T}()
    for i in eachindex(e.vars)
        v, c = e.vars[i], e.coeffs[i]
        if c != 0
            coeffs[v] = get(coeffs, v, zero(T)) + c
        end
    end
    pairs = collect(coeffs)
    sort!(pairs)
    AffExpr(first.(pairs), last.(pairs), e.constant)
end

Conditional(::typeof(<=), x, y) = Conditional(Set([Constraint{Float64}(simplify(x - y), -Inf, 0)]))
Conditional(::typeof(>=), x, y) = Conditional(Set([Constraint{Float64}(simplify(y - x), -Inf, 0)]))

# Conditional(::typeof(<=), x, y) = Set([Constraint(simplify(x - y), false)])
# Conditional(::typeof(>=), x, y) = Set([Constraint(simplify(y - x), false)])
# Conditional(::typeof(==), x, y) = Set([Constraint(simplify(x - y), true)])
# (&)(c1::Conditional, c2::Conditional) = union(c1, c2)

nan_to_null(x) = isnan(x) ? Nullable{typeof(x)}() : Nullable{typeof(x)}(x)

"""
Like JuMP.getvalue, but returns a Nullable{T}() for unset variables instead
of throwing a warning
"""
_getvalue(x::Variable) = nan_to_null(JuMP._getValue(x))
_getvalue(x::Number) = nan_to_null(x)

function _getvalue(x::JuMP.GenericAffExpr{T, Variable}) where {T}
    result::Nullable{T} = x.constant
    for i in eachindex(x.coeffs)
        result = result .+ x.coeffs[i] .* _getvalue(x.vars[i])
    end
    result
end

function _getvalue(h::Constraint)
    v = _getvalue(h.terms)
    if h.equality
        return abs.(v)
    else
        return v
    end
end

# _getvalue(c::Conditional) = maximum(_getvalue, c.args)

# hascomplement(c::Conditional) = (length(c) == 1 && 
# function complement(c::Conditional)
#     @assert hascomplement(c)
#     Conditional([Constraint(-first(c).expr)]))
# end

# show(io::IO, h::HashableAffExpr) = print(io, h.expr)

# function show(io::IO, c::Conditional)
#     print(io, join(["($arg <= 0)" for arg in c.args], " & "))
# end

end
    

cj3

In [94]:
m = Model()
@variable m x
c1 = cj3.Conditional(<=, x, 0)
c2 = cj3.Conditional(>=, x, 0)
c3 = cj3.Conditional(<=, x + 5, 5)

@test c1 != c2
@test c2 != c3
@test c1 == c3
@test c1 !== c3
@test hash(c1) == hash(c3)
@test hash(c1) != hash(c2)
@test isnull(cj3._getvalue(c1))
@test isnull(cj3._getvalue(c2))
setvalue(x, -0.1)
@test get(cj3._getvalue(c1)) == -0.1
@test get(cj3._getvalue(c2)) == 0.1

Test Failed
  Expression: c1 == c3
   Evaluated: cj3.Conditional(Set(JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}[x ≤ 0])) == cj3.Conditional(Set(JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}[x ≤ 0]))


LoadError: [91mThere was an error during testing[39m

In [101]:
@edit first(c1.constraints) == first(c3.constraints)

In [100]:
dump(first(c1.constraints))

JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}
  terms: JuMP.GenericAffExpr{Float64,JuMP.Variable}
    vars: Array{JuMP.Variable}((1,))
      1: JuMP.Variable
        m: JuMP.Model
          obj: JuMP.GenericQuadExpr{Float64,JuMP.Variable}
            qvars1: Array{JuMP.Variable}((0,))
            qvars2: Array{JuMP.Variable}((0,))
            qcoeffs: Array{Float64}((0,)) Float64[]
            aff: JuMP.GenericAffExpr{Float64,JuMP.Variable}
              vars: Array{JuMP.Variable}((0,))
              coeffs: Array{Float64}((0,)) Float64[]
              constant: Float64 0.0
          objSense: Symbol Min
          linconstr: Array{JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}}((0,))
          quadconstr: Array{JuMP.GenericQuadConstraint{JuMP.GenericQuadExpr{Float64,JuMP.Variable}}}((0,))
          sosconstr: Array{JuMP.SOSConstraint}((0,))
          socconstr: Array{JuMP.GenericSOCConstraint{JuMP.GenericNormExpr{2,Float64,JuMP.Variabl

In [96]:
c3.constraints

Set(JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}[x ≤ 0])

In [75]:
c1 & c3

(x <= 0)

In [76]:
c1 & c2

(-x <= 0) & (x <= 0)

In [77]:
cj3.complement(c1)

(-x <= 0)

In [78]:
@benchmark $c1 == $c2

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     102.877 ns (0.00% GC)
  median time:      104.687 ns (0.00% GC)
  mean time:        113.770 ns (0.00% GC)
  maximum time:     453.154 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     937

In [43]:
@benchmark $c1 == $c3

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  1
  --------------
  minimum time:     166.368 ns (0.00% GC)
  median time:      169.091 ns (0.00% GC)
  mean time:        191.204 ns (1.22% GC)
  maximum time:     3.060 μs (87.32% GC)
  --------------
  samples:          10000
  evals/sample:     750

In [36]:
c1 == c3

true

In [22]:
c1.args

Set(cj3.HashableAffExpr[cj3.HashableAffExpr(x)])

In [24]:
c3.args

Set(cj3.HashableAffExpr[cj3.HashableAffExpr(x)])

In [26]:
c1.args == c3.args

false

In [31]:
@edit (x + 1) == (x - 1)

In [30]:
@edit first(c1.args) == first(c3.args)

In [28]:
first(c3.args)

cj3.HashableAffExpr(x)